In [1]:
"""Mini projet sur les données de valeurs foncières en France"""
################################################################################
# fichier  : pretraitement.py
# Auteur : Rakotoson Loïc
################################################################################

################################################################################
# Importation de fonctions externes :
import numpy as np
import pickle
import json
import csv
from numba import jit, cuda
from os import listdir
from datetime import datetime

################################################################################
# Definition locale de fonctions :

In [2]:
# Récupération des variables
def readData(nomFichier):
    listeData = []
    try:
        with open(nomFichier, 'rb') as objet_fichier:
            objet_fichier_depickler = pickle.Unpickler(objet_fichier)
            while 1:
                try:
                    listeData.append(objet_fichier_depickler.load())
                except EOFError:
                    break
    except (IOError, OSError):
        print("Problème à l'ouverture du fichier")
    return listeData

In [3]:
# Importation des données
def lectureArray(path, filestart, listeannee, ncol=43):
    """
    filestart: nom commun des fichiers
    :return: array
    """
    i = len(filestart)
    listfiles = [
        path + files for files in listdir(path)
        if (files[:i] == filestart and files[i + 1:i + 5] in listeannee)
    ]
    header = np.genfromtxt(listfiles[0], delimiter="|", dtype=str,
                           max_rows=1).reshape((1, ncol))
    data = np.concatenate([
        np.genfromtxt(file, delimiter="|", dtype=str, skip_header=True)
        for file in listfiles
    ])
    data = np.concatenate((header, data), axis=0)
    return data

In [4]:
# Calcul centre de GPS
@jit
def centreGPS(gps):
    angle = gps*np.pi/180
    
    x = np.sum(np.cos(angle[:,0]).T*np.cos(angle[:,1])) / np.shape(gps)[0]
    y = np.sum(np.cos(angle[:,0]).T*np.sin(angle[:,1])) / np.shape(gps)[0]
    z = np.sum(np.sin(angle[:,0])) / np.shape(gps)[0]
    
    lon = np.arctan2(x,y)
    hyp = np.sqrt(x**2 + y**2)
    lat = np.arctan2(z, hyp)
    
    lat = lat * 180/np.pi
    lon = lon * 180/np.pi
    
    return [lat, lon]

In [5]:
################################################################################
# Definition de classe

################################################################################
#Récupération des données

#reponses = readData('donnees/reponses.pic')
#La variable reponses représente l'ensemble des réponses aux questions du projet
#Aide_ex14 = reponses[0]
#Aide_ex21 = reponses[1]
#Aide_ex22 = reponses[2]
#Aide_ex32 = reponses[3]
################################################################################
# Corps principal du programme :

# Pretraitement
## Exercice 1 : Lecture des données de valeurs foncières
Faire une lecture d’un fichier de valeurs foncières (« valeursfoncieres-20XX_rennes_metropole.txt»). Le format du fichier est de type texte et le séparateur de champ est le symbole «| » (ce fichier peut-être vu comme un fichier « csv »)

In [6]:
data = lectureArray("donnees/", "valeursfoncieres",
                    ("2017"))
data = data.tolist() #Transformer en liste car memory error à l'exo 2.2

## Exercice 2 : Lecture des données cadastrales des parcelles
Le rôle de ce fichier est de situer chaque bien par sa parcelle. Chaque bien des fichiers de valeursfoncières est identifié par ses références cadastrales (« Code departement », « Code commune », «Prefixe de section », « Section », « No plan »).

Faire une lecture du fichier de données cadastrales en faisant en sorte d’obtenir un dictionnairedont la clé correspondra aux références cadastrales (clé : (commune, préfixe, section, numéro) ;Attention dans ce fichier « Code departement » et « Code commune » sont regroupés sous le nom « commune ») et la valeur correspondra aux coordonnées géographiques de la parcelle.

In [7]:
with open("donnees/cadastre.csv", 'r', encoding="utf-8") as file:
    cadastre = [ligne for ligne in csv.reader(file, delimiter=";")]

elt = [
    cadastre[0].index(c)
    for c in ("commune", "prefixe", "section", "numero", "coordinates")
]
cadastre = {(row[elt[0]], row[elt[1]], row[elt[2]], row[elt[3]]): row[elt[4]]
            for row in cadastre[1:]}

Insérer une nouvelle colonne dans les données de valeurs foncières de Rennes métropole.Cette colonne contiendra les coordonnées GPS de la parcelle de chaque bien. Si les références cadastrales du bien sont introuvables, on ajoutera une chaîne de caractère vide.

In [8]:
for row in data[1:]:
    try:
        row.append(
            np.array(
                json.loads(cadastre[(row[18] + row[19], '000', row[21],
                                     row[22])])[0]))
    except KeyError:
        row.append("")

data[0].append("GPS")  # Header

## Exercice 3 : Calcul sur les données cadastrales.
Créer une fonction permettant de calculer le point milieu (coordonnées GPS) de chaque 
parcelle.

Créer un vecteur (liste ou array Numpy) contenant le point milieu de chaque parcelle (Attention le vecteur et les données de valeurs foncières doivent être alignés).

In [9]:
cgps = np.array([centreGPS(row[43]) if row[43] !="" else [np.nan,np.nan] for row in data[1:]])

C:\Users\SIGMA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.
C:\Users\SIGMA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


## Exercice 4 : Sauvegarde des nouvelles données
Enregistrer dans un fichier « csv » les données de valeurs foncières.

In [10]:
with open('valeursfonciere.csv','w') as result_file:
    wr = csv.writer(result_file, delimiter = ",")
    wr.writerows(data)

Enregistrer dans un fichier « csv » le vecteur des coordonnées GPS des biens.

In [11]:
np.savetxt('centregps.csv', cgps, delimiter = ",")